In [43]:

import nltk
import re
%matplotlib inline
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.neural_network import MLPClassifier


from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import FunctionTransformer

In [44]:
df0=pd.read_csv('scraped_data/crawling_0.csv')
df1=pd.read_csv('scraped_data/crawling_1.csv')
df2=pd.read_csv('scraped_data/crawling_2.csv')
df3=pd.read_csv('scraped_data/crawling_3.csv')
df4=pd.read_csv('scraped_data/crawling_4.csv')
df5=pd.read_csv('scraped_data/crawling_5.csv')
'''

df6=pd.read_csv('scraped_data/crawling_6.csv')
df7=pd.read_csv('scraped_data/crawling_7.csv')
df8=pd.read_csv('scraped_data/crawling_8.csv')
df9=pd.read_csv('scraped_data/crawling_9.csv')
df10=pd.read_csv('scraped_data/crawling_10.csv')
df11=pd.read_csv('scraped_data/crawling_11.csv')
data_df=pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11])
'''
data_df=pd.concat([df0,df1,df2,df3,df4,df5])

In [45]:
data_df.reset_index(drop=True, inplace=True)

In [46]:
selected_categories = data_df['class'].value_counts()[:10].index.tolist()
selected_categories

['Thời sự',
 'Thế giới',
 'Thể thao',
 'Pháp luật',
 'Sức khỏe',
 'Kinh doanh',
 'Cần biết',
 'Giáo dục',
 'Nhịp sống trẻ',
 'Văn hóa']

In [47]:
data_df = data_df[data_df['class'].isin(selected_categories)]
data_df

,links,title,description,content,class
0,https://tuoitre.vn/tong-thong-trump-xac-nhan-k...,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,https://tuoitre.vn/dat-nuoc-dat-niem-tin-vao-n...,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,https://tuoitre.vn/luat-su-my-phan-bien-ong-tr...,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,https://tuoitre.vn/thanh-pho-phu-quoc-se-phat-...,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,https://tuoitre.vn/ong-trump-nguoi-ung-ho-toi-...,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...,...
27993,/quy-do-thang-hoa-bang-bang-vao-top-4-co-co-ho...,"'Quỷ đỏ' thăng hoa băng băng vào top 4, có cơ ...","TTO - Giành hai danh hiệu, lọt vào top 4 Giải ...","Nửa năm trước, cựu danh thủ người Na Uy còn nằ...",Thể thao
27995,/bao-chi-can-len-an-hanh-vi-suy-thoai-cua-mot-...,Báo chí cần lên án hành vi suy thoái của một s...,TTO - Đó là đề nghị của Thứ trưởng Bộ Thông ti...,Hội nghị tập huấn bồi dưỡng kiến thức về xây d...,Thời sự
27996,/kho-hang-lau-khung-tai-lao-cai-thu-tien-ti-tu...,Kho hàng lậu khủng tại Lào Cai: Thu tiền tỉ từ...,"TTO - Nhiều sản phẩm thời trang, quần áo, giày...","Đây là vụ kinh doanh hàng giả, hàng lậu online...",Pháp luật
27997,/ban-giai-phap-go-kho-thuc-kinh-te-tp-hcm-phat...,Bàn giải pháp gỡ khó thúc kinh tế TP.HCM phát ...,"TTO - Sáng 9-7, kỳ họp thứ 20 của HĐND TP.HCM ...","Phát biểu khai mạc kỳ họp, sẽ diễn ra trong 3 ...",Thời sự


In [48]:
data_df=data_df[~((data_df['title'].isnull())|
                  (data_df['description'].isnull())|
                  (data_df['content'].isnull()))]

In [49]:
data_df.drop(['links'],axis=1,inplace=True)

C:\Users\Admin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [50]:
data_df.drop_duplicates(inplace=True)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
data_df

,title,description,content,class
0,Tổng thống Trump xác nhận không dự lễ nhậm chứ...,TTO - Tổng thống Mỹ Donald Trump đăng tweet ch...,Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao ...,Thế giới
1,Đất nước đặt niềm tin vào những học sinh xuất sắc,"TT0 - Tối 8-1, Thủ tướng Chính phủ Nguyễn Xuân...","Chia sẻ tại buổi lễ, Thủ tướng Nguyễn Xuân Phú...",Giáo dục
2,Luật sư Mỹ phản biện: Ông Trump đâu có kêu ngư...,TTO - Trên chương trình Bill Hemmer Reports củ...,"Ngày 6-1, tình trạng bạo lực đã xảy ra tại tòa...",Thế giới
3,Thành phố Phú Quốc sẽ phát triển dựa trên 4 tr...,"TTO - Tối 8-1 tại phường An Thới, chính quyền ...",Phát biểu tại buổi lễ công bố thành lập TP Phú...,Thời sự
4,Ông Trump: Người ủng hộ tôi 'sẽ có tiếng nói t...,TTO - Trong nội dung đăng trên Twitter sau gần...,"""75 triệu người Mỹ yêu nước vĩ đại đã bầu cho ...",Thế giới
...,...,...,...,...
27993,"'Quỷ đỏ' thăng hoa băng băng vào top 4, có cơ ...","TTO - Giành hai danh hiệu, lọt vào top 4 Giải ...","Nửa năm trước, cựu danh thủ người Na Uy còn nằ...",Thể thao
27995,Báo chí cần lên án hành vi suy thoái của một s...,TTO - Đó là đề nghị của Thứ trưởng Bộ Thông ti...,Hội nghị tập huấn bồi dưỡng kiến thức về xây d...,Thời sự
27996,Kho hàng lậu khủng tại Lào Cai: Thu tiền tỉ từ...,"TTO - Nhiều sản phẩm thời trang, quần áo, giày...","Đây là vụ kinh doanh hàng giả, hàng lậu online...",Pháp luật
27997,Bàn giải pháp gỡ khó thúc kinh tế TP.HCM phát ...,"TTO - Sáng 9-7, kỳ họp thứ 20 của HĐND TP.HCM ...","Phát biểu khai mạc kỳ họp, sẽ diễn ra trong 3 ...",Thời sự


In [52]:
from pyvi import ViTokenizer, ViPosTagger # thư viện NLP tiếng Việt
from tqdm import tqdm
import numpy as np
import gensim # thư viện NLP

In [53]:
data_df["content"][0]

'Sau khi đưa ra cam kết sẽ đảm bảo chuyển giao quyền lực một cách hòa bình, êm thắm, ngày 8-1, ông Trump viết trên Twitter: "Trả lời câu hỏi của các bạn, tôi sẽ không tham dự lễ nhậm chức ngày 20-1".Dòng tweet này của ông liên tục biến động các con số về lượt người xem và chia sẻ cũng như bình luận. Nhiều người phản đối đã để lại bình luận không mấy thân thiện bên dưới như "không ai muốn ngài có mặt ở đó"...Theo Hãng tin Reuters, một nguồn tin am hiểu nội vụ cho biết ở Nhà Trắng đã có sự bàn bạc về việc ông Trump sẽ rời Washington ngày 19-1. Cũng theo nguồn tin này, ông Trump sẽ đến khu nghỉ dưỡng của mình ở bang Florida.Trong lịch sử Mỹ, có nhiều Tổng thống từng quyết định không dự lễ nhậm chức của người kế nhiệm.\xa0Tổng thống Richard Nixon rời Nhà Trắng khi Gerald Ford tuyên thệ. Ông Nixon từ chức, báo với Phó tổng thống của mình là Gerald Ford vào ngày 8-8 và ông Ford thay mình từ "ngày hôm nay".\xa0 Lúc 9h30 ngày 9-8, ông Nixon có bài phát biểu cuối cùng sau đó rời đi. Khi ông đáp

In [54]:
#' '.join(gensim.utils.simple_preprocess(data_df['content'][0]))
def gensim_utils_simple_preprocess(x):
    return ' '.join(gensim.utils.simple_preprocess(x))
data_df['content']=data_df['content'].apply(gensim_utils_simple_preprocess)
data_df['class']=data_df['class'].apply(gensim_utils_simple_preprocess)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [55]:
def chuan_hoa_dau_tu_tieng_viet(word):
    if not is_valid_vietnam_word(word):
        return word
 
    chars = list(word)
    dau_cau = 0
    nguyen_am_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            dau_cau = y
            chars[index] = bang_nguyen_am[x][0]
        if not qu_or_gi or index != 1:
            nguyen_am_index.append(index)
    if len(nguyen_am_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = nguyen_am_to_ids.get(chars[1])
                chars[1] = bang_nguyen_am[x][dau_cau]
            else:
                x, y = nguyen_am_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = bang_nguyen_am[x][dau_cau]
                else:
                    chars[1] = bang_nguyen_am[5][dau_cau] if chars[1] == 'i' else bang_nguyen_am[9][dau_cau]
            return ''.join(chars)
        return word
 
    for index in nguyen_am_index:
        x, y = nguyen_am_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = bang_nguyen_am[x][dau_cau]
            # for index2 in nguyen_am_index:
            #     if index2 != index:
            #         x, y = nguyen_am_to_ids[chars[index]]
            #         chars[index2] = bang_nguyen_am[x][0]
            return ''.join(chars)
 
    if len(nguyen_am_index) == 2:
        if nguyen_am_index[-1] == len(chars) - 1:
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            chars[nguyen_am_index[0]] = bang_nguyen_am[x][dau_cau]
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            # chars[nguyen_am_index[1]] = bang_nguyen_am[x][0]
        else:
            # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
            # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
            x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
            chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
    else:
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[0]]]
        # chars[nguyen_am_index[0]] = bang_nguyen_am[x][0]
        x, y = nguyen_am_to_ids[chars[nguyen_am_index[1]]]
        chars[nguyen_am_index[1]] = bang_nguyen_am[x][dau_cau]
        # x, y = nguyen_am_to_ids[chars[nguyen_am_index[2]]]
        # chars[nguyen_am_index[2]] = bang_nguyen_am[x][0]
    return ''.join(chars)

def is_valid_vietnam_word(word):
    chars = list(word)
    nguyen_am_index = -1
    for index, char in enumerate(chars):
        x, y = nguyen_am_to_ids.get(char, (-1, -1))
        if x != -1:
            if nguyen_am_index == -1:
                nguyen_am_index = index
            else:
                if index - nguyen_am_index != 1:
                    return False
                nguyen_am_index = index
    return True

def chuan_hoa_dau_cau_tieng_viet(sentence):
    """
        Chuyển câu tiếng việt về chuẩn gõ dấu kiểu cũ.
        :param sentence:
        :return:
        """
    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\\p{P}*)([p{L}.]*\\p{L}+)(\\p{P}*$)', r'\\1/\\2/\\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = chuan_hoa_dau_tu_tieng_viet(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)



data_df['content']=data_df['content'].apply(chuan_hoa_dau_cau_tieng_viet)
data_df['class']=data_df['class'].apply(chuan_hoa_dau_cau_tieng_viet)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic
dicchar = loaddicchar()
 
# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

data_df['content']=data_df['content'].apply(covert_unicode)
data_df['class']=data_df['class'].apply(covert_unicode)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [57]:


def lower_character(a):
    return a.lower()
data_df['content']=data_df['content'].apply(lower_character)
#xoá ký tự thừa
def remove_unnecessary(paragraph):
    temp=re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',paragraph)
    return re.sub(r'\s+', ' ', temp).strip()

data_df['content']=data_df['content'].apply(remove_unnecessary)


C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# Tokenize


In [58]:
data_df['content']=data_df['content'].apply(ViTokenizer.tokenize)

C:\Users\Admin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


# Sau khi tìm hiểu xong ta bắt đầu chia tập dữ liệu và bắt đầu tiền xử lí

In [59]:
y_sr = data_df["class"] # sr là viết tắt của series
X_df = data_df["content"]
train_X_df, val_X_df, train_y_sr, val_y_sr = train_test_split(X_df, y_sr, test_size=0.3,stratify=y_sr, random_state=0)

0        sau khi đưa ra cam_kết sẽ đảm_bảo chuyển_giao ...
1        chia_sẻ tại buổi lễ thủ_tướng nguyễn xuân phúc...
2        ngày tình_trạng bạo_lực đã xảy ra tại tòa nhà ...
3        phát_biểu tại buổi lễ công_bố thành_lập tp phú...
4        triệu người mỹ yêu nước vĩ_đại đã bầu cho tôi ...
                               ...                        
27993    nửa năm trước cựu danh_thủ người na uy còn nằm...
27995    hội_nghị tập_huấn bồi_dưỡng kiến_thức về xây_d...
27996    đây là vụ kinh_doanh hàng giả hàng lậu online ...
27997    phát_biểu khai_mạc kỳ họp sẽ diễn ra trong ngà...
27998    đó là lời khẳng_định của ts phan thanh hải giá...
Name: content, Length: 19199, dtype: object

# Chiết xuất stopwords ở từng cell content với TFIDF

In [61]:
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
everything_in_one_big_str =[" ".join(" ".join(data_df['content'].tolist()).split(' '))]
freq=count_vect.fit_transform(everything_in_one_big_str)
count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
                vocabulary=None)

In [34]:
count_vect.get_feature_names()

37912

In [63]:
 data_df['content'][3]

'phát_biểu tại buổi lễ công_bố thành_lập tp phú quốc phó thủ_tướng thường_trực chính_phủ trương hòa bình biểu_dương những nỗ_lực cố_gắng của tỉnh kiên_giang và tp phú quốc thời_gian qua đã đưa phú quốc từ một hòn đảo còn nhiều thiếu_thốn vươn tầm phát_triển mạnh_mẽ theo phó thủ_tướng phú quốc đã có sân_bay cảng biển có điện_lưới quốc_gia có nhiều tập_đoàn lớn đầu_tư nhiều công_trình quy_mô trang_bị hiện_đại đạt tầm khu_vực và thế_giới phú quốc đã có hình_hài một đô_thị thông_minh hiện_đại được nhiều du_khách trong và ngoài nước biết tới cùng với sự phát_triển kinh_tế xã_hội quá_trình phát_triển đô_thị trên địa_bàn phú quốc diễn ra nhanh đã đặt ra yêu_cầu mới cần_thiết phải có một bộ_máy chính_quyền đô_thị có trình_độ phát_triển cao hiện_đại đủ năng_lực thực_thi có hiệu_quả công_tác quản_lý hành_chính nhà_nước trên địa_bàn phó thủ_tướng trương hòa_bình dặn_dò chính_quyền tỉnh kiên giang tp phú quốc tập_trung_thực_hiện một_số nhiệm_vụ trọng_tâm trong đó có việc quy_hoạch phát_triển phú q

In [67]:
'''
tfidf = TfidfTransformer(norm="l2")
tfidf.fit(freq_term_matrix)  

feature_names = count_vect.get_feature_names()

doc = data_df['content'][3]

tf_idf_vector = tfidf.transform(count_vect.transform([doc]))

coo_matrix = tf_idf_vector.tocoo()
tuples = zip(coo_matrix.col, coo_matrix.data)
sorted_items = sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

# extract only the top n elements.
# Here, n is 10.
word_tfidf = extract_topn_from_vector(feature_names,sorted_items,10)

print("{}  {}".format("features", "tfidf"))  
for k in word_tfidf:
    print("{} - {}".format(k[0], k[1])) 

'''
from sklearn.feature_extraction.text import TfidfTransformer
def extract_topn_from_vector(feature_names, sorted_items, topn=5):
    """
      get the feature names and tf-idf score of top n items in the doc,                 
      in descending order of scores. 
    """

    # use only top n items from vector.
    sorted_items = sorted_items[:topn]

    results= {} 
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        results[feature_names[idx]] = round(score, 3)

    # return a sorted list of tuples with feature name and tf-idf score as its element(in descending order of tf-idf scores).
    return sorted(results.items(), key=lambda kv: kv[1], reverse=True)

count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
everything_in_one_big_str =[" ".join(" ".join(data_df['content'].tolist()).split(' '))]
freq_term_matrix = count_vect.fit_transform(everything_in_one_big_str)

freq_term_matrix=freq_term_matrix.toarray()
extract_topn_from_vector()





array([[2, 2, 1, ..., 1, 1, 3]], dtype=int64)

In [19]:
import nltk
from nltk.corpus import stopwords
print(stopwords.words('english'))

def remove_stopwords(line):
    words = []
    for word in line.strip().split():
        if word not in stopword:
            words.append(word)
    return ' '.join(words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [40]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(train_X_df)

# transform the training and validation data using count vectorizer object
X_data_count = count_vect.transform(train_X_df)
X_test_count = count_vect.transform(val_X_df)

In [41]:
# word level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect = TfidfVectorizer(analyzer='word', max_features=30000)
tfidf_vect.fit(train_X_df) # learn vocabulary and idf from training set
X_data_tfidf =  tfidf_vect.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf =  tfidf_vect.transform(val_X_df)

In [43]:
# ngram level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram.fit(train_X_df)
X_data_tfidf_ngram =  tfidf_vect_ngram.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf_ngram =  tfidf_vect_ngram.transform(val_X_df)

In [44]:
# ngram-char level - we choose max number of words equal to 30000 except all words (100k+ words)
tfidf_vect_ngram_char = TfidfVectorizer(analyzer='char', max_features=30000, ngram_range=(2, 3))
tfidf_vect_ngram_char.fit(train_X_df)
X_data_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(train_X_df)
# assume that we don't have test set before
X_test_tfidf_ngram_char =  tfidf_vect_ngram_char.transform(val_X_df)

In [45]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=300, random_state=42)
svd.fit(X_data_tfidf)


X_data_tfidf_svd = svd.transform(X_data_tfidf)
X_test_tfidf_svd = svd.transform(X_test_tfidf)

In [46]:
svd_ngram = TruncatedSVD(n_components=300, random_state=42)
svd_ngram.fit(X_data_tfidf_ngram)

X_data_tfidf_ngram_svd = svd_ngram.transform(X_data_tfidf_ngram)
X_test_tfidf_ngram_svd = svd_ngram.transform(X_test_tfidf_ngram)

In [47]:
svd_ngram_char = TruncatedSVD(n_components=300, random_state=42)
svd_ngram_char.fit(X_data_tfidf_ngram_char)

X_data_tfidf_ngram_char_svd = svd_ngram_char.transform(X_data_tfidf_ngram_char)
X_test_tfidf_ngram_char_svd = svd_ngram_char.transform(X_test_tfidf_ngram_char)